In [10]:
import pandas

salaries = pandas.read_csv("salaries_clean.csv", encoding = "ISO-8859-1")

salaries.head(10)

,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at
0,1,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,NaN,13.0,2.0,125000.0,5000.0,0.0,5000 shares,Don't work here.,3/21/16 12:58
1,3,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",NaN,3/21/16 12:58
2,4,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,NaN,4.0,1.0,86000.0,5000.0,6000.0,0,NaN,3/21/16 12:59
3,6,netapp,waltham,NaN,NaN,NaN,NaN,mts,Other,NaN,4.0,0.0,105000.0,5000.0,8500.0,0,NaN,3/21/16 13:00
4,12,apple,cupertino,NaN,NaN,NaN,NaN,software engineer,Software,NaN,4.0,3.0,110000.0,5000.0,7000.0,150000,NaN,3/21/16 13:02
5,14,casino,eastern oregon,OR,US,38.00,-97.00,it technician,Other,NaN,5.0,1.5,40000.0,0.0,500.0,0,NaN,3/21/16 13:03
6,16,tetra tech,"madison, wi",WI,US,43.10,-89.50,lead interviewer,Other,NaN,4.0,2.5,45000.0,0.0,1500.0,NaN,NaN,3/21/16 13:03
7,17,knewton,new york city,NY,US,38.00,-97.00,senior software engineer,Software,Senior,8.0,2.0,135000.0,0.0,0.0,NaN,NaN,3/21/16 13:03
8,21,amplify,nyc,NaN,NaN,NaN,NaN,software engineer,Software,NaN,2.0,1.0,105000.0,0.0,47000.0,0,NaN,3/21/16 13:04
9,23,sqor,san francisco,NaN,NaN,NaN,NaN,infrastructure/platform engineer,Engineering,NaN,1.0,1.0,80000.0,0.0,0.0,500,NaN,3/21/16 13:04


# Data Cleansing
- Clean Locations, Shares, and other random variables
- Drop comments
- Add Total Comp Column (Exlcuding Shares)

# Data Analysis
- Salary based on location, years of experience, country
- Calculate most recorded companies
- Percent difference between ranks
- Possibly set premade ranks based on years of experience

# Machine Learning
- Salary increase based on rank over time
- Salary predictor
- Rank based on years of experience